In [1]:
import pandas as pd
import csv
import psycopg2
import logging
import config

In [2]:
host="localhost"
database="ds"


def create_connection(db_file):    
    conn = None
    try:
        conn = psycopg2.connect(host=host,
                        database=database,
                        user=config.user,
                        password=config.password
                        )         
        print(psycopg2.__version__)
    except IOError  as e:
        print(e) 
    return conn

In [3]:
def drop_Resources_table(conn):
    
    cur = conn.cursor()    
    drop_Resources_table = """
        DROP TABLE Resources;               
        """
    cur.execute(drop_Resources_table)    
 

In [4]:
def create_Resources_table(conn):
    
    cur = conn.cursor()    
    create_Resources_table = """
        
        CREATE TABLE IF NOT EXISTS Resources (
            Id          text PRIMARY KEY NOT NULL,
            Link        Text,
            Title       Text,
            Description Text  ) 
        
        """
    cur.execute(create_Resources_table)    
 

In [5]:
def import_from_csv_to_table(conn):
    
    cur = conn.cursor()    
    data = None
    try:
        
        data =  open('data/clean_resource.csv', 'r')
        columns = data.readline().strip('\n').split(',')      
        cur.copy_from(data, 'Resources', sep=',', columns=('id','link', 'title', 'description'))     
        data.close()
        
    except IOError:
        logging.exception('')  
        
    if not data:
        raise ValueError('No data available')

In [6]:
def select_2_rows_resource(conn):
    
    cur = conn.cursor()
    
    query1 = """
        SELECT * FROM Resources limit 2         
        """
    cur.execute(query1) 
    rows = cur.fetchall()
 
    for row in rows:
          print(f"Id: {row[0]}\nLink:{row[1]}\nTitle:{row[2]}\nDescription: {row[3]}")

In [7]:
def main():
    # create a database connection
    conn = create_connection(database)
    with conn: 
        drop_Resources_table(conn) 
        
        create_Resources_table(conn)        
        
        import_from_csv_to_table(conn)        
        
        print("Here is 2 rows of table:")
        select_2_rows_resource(conn)
        
    conn.commit()
    conn.close()    
 
if __name__ == '__main__':
    main()

2.8.6 (dt dec pq3 ext lo64)
Here is 2 rows of table:
Id: 06219
Link:http://arxiv.org/abs/2102.06219
Title: Silentium! Run-Analyse-Eradicate the Noise out of the DB/OS Stack
Description: When multiple tenants compete for resources  database performance tends tosuffer. Yet there are scenarios where guaranteed sub-millisecond latencies arecrucial  such as in real-time data processing  IoT devices  or when operatingin safety-critical environments. In this paper  we study how to make querylatencies deterministic in the face of noise (whether caused by other tenantsor unrelated operating system tasks). We perform controlled experiments with anin-memory database engine in a multi-tenant setting  where we successivelyeradicate noisy interference from within the system software stack  to thepoint where the engine runs close to bare-metal on the underlying hardware.We show that we can achieve query latencies comparable to the database enginerunning as the sole tenant  but without noticeably impa